In [ ]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split

#1. data
path = 'D:/study_data/_data/dacon_travel/'
train_set = pd.read_csv(path + 'train.csv', index_col=0)
test_set = pd.read_csv(path + 'test.csv', index_col=0)

# print(train_set)
# print(train_set.shape) # (1955, 19)
# print(train_set.columns)
# Index(['Age', 'TypeofContact', 'CityTier', 'DurationOfPitch', 'Occupation',
#        'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
#        'ProductPitched', 'PreferredPropertyStar', 'MaritalStatus',
#        'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar',
#        'NumberOfChildrenVisiting', 'Designation', 'MonthlyIncome',
#        'ProdTaken'],
#       dtype='object')

# print(train_set.info())
# print(train_set.describe())

# print(test_set)
# print(test_set.shape) # (2933, 18)

# 결측치 확인
# print(train_set.isnull().sum())
# print(test_set.isnull().sum())

데이터를 불러와서 데이터의 상세내용을 확인한다. info, describe, features 및 결측치를 확인한다.

In [ ]:
# 결측치 처리
# DurationOfPitch 결측치 0으로 채우기
train_set['DurationOfPitch'] = train_set['DurationOfPitch'].fillna(0)
test_set['DurationOfPitch'] = test_set['DurationOfPitch'].fillna(0)

# TypeofContact 결측치 "Self Enquiry"로 채우기
train_set['TypeofContact'] = train_set['TypeofContact'].fillna("Self Enquiry")
test_set['TypeofContact'] = test_set['TypeofContact'].fillna("Self Enquiry")

# Gender의 Fe male -> Female 로 변경
# print(train_set['Gender'].value_counts())
# train_set['Gender'] = train_set.replace({'Gender' : 'Fe Male'}, 'Female') # df = df.replace({'열 이름' : 기존 값}, 변경 값) 
# test_set['Gender'] = test_set.replace({'Gender' : 'Fe Male'}, 'Female') # ValueError: Columns must be same length as key [원인찾기]
train_set['Gender'] = train_set['Gender'].str.replace('Fe Male', 'Female')
test_set['Gender'] = train_set['Gender'].str.replace('Fe Male', 'Female')

# MonthlyIncome (Designation별 평균으로 변경)
train_set['MonthlyIncome'] = train_set['MonthlyIncome'].fillna(train_set.groupby('Designation')['MonthlyIncome'].transform('median'))
test_set['MonthlyIncome'] = test_set['MonthlyIncome'].fillna(test_set.groupby('Designation')['MonthlyIncome'].transform('median'))

# Occupation / freelancer -> 최빈값(Salaried)으로 변경
# print(train_set['Occupation'].value_counts())
# train_set['Occupation'] = train_set['Occupation'].str.replace('Free Lancer', train_set['Occupation'].mode())
train_set['Occupation'] = train_set['Occupation'].str.replace('Free Lancer', 'Salaried')
test_set['Occupation'] = test_set['Occupation'].str.replace('Free Lancer', 'Salaried')

# MarialStatus의 Unmarried, Divorced -> Single로 변경 등 여러가지 경우의 수로 맵핑 진행
# print(train_set['MaritalStatus'].value_counts())
# Married      949
# Divorced     375
# Single       349
# Unmarried    282
# Name: MaritalStatus, dtype: int64
# train_set['MaritalStatus'] = train_set['MaritalStatus'].str.replace('Divorced', 'Single')
# test_set['MaritalStatus'] = test_set['MaritalStatus'].str.replace('Divorced', 'Single')
# train_set['MaritalStatus'] = train_set['MaritalStatus'].str.replace('Unmarried', 'Single')
# test_set['MaritalStatus'] = test_set['MaritalStatus'].str.replace('Unmarried', 'Single')

결측치 확인 후 컬럼별로 적합하다고 생각되는 전처리를 진행한다.

마지막 MarialStatus의 경우 다양한 방법으로 맵핑을 진행해보았으나 그대로 두는 것이 가장 성능이 좋은 것을 확인했다.

In [ ]:
train_set['NumberOfFollowups'].fillna(train_set.groupby('NumberOfChildrenVisiting')['NumberOfFollowups'].transform('mean'), inplace=True)
test_set['NumberOfFollowups'].fillna(test_set.groupby('NumberOfChildrenVisiting')['NumberOfFollowups'].transform('mean'), inplace=True)

train_set['PreferredPropertyStar'].fillna(train_set.groupby('Occupation')['PreferredPropertyStar'].transform('mean'), inplace=True)
test_set['PreferredPropertyStar'].fillna(test_set.groupby('Occupation')['PreferredPropertyStar'].transform('mean'), inplace=True)

train_set['NumberOfTrips'].fillna(train_set.groupby('DurationOfPitch')['NumberOfTrips'].transform('mean'), inplace=True)
test_set['NumberOfTrips'].fillna(test_set.groupby('DurationOfPitch')['NumberOfTrips'].transform('mean'), inplace=True)

train_set['NumberOfChildrenVisiting'].fillna(train_set.groupby('MaritalStatus')['NumberOfChildrenVisiting'].transform('mean'), inplace=True)
test_set['NumberOfChildrenVisiting'].fillna(test_set.groupby('MaritalStatus')['NumberOfChildrenVisiting'].transform('mean'), inplace=True)

# train_set['AgeBand'] = pd.cut(train_set['Age'], 5)
# 임의로 5개 그룹을 지정
# print(train_set['AgeBand'])
# [(17.957, 26.6] < (26.6, 35.2] < (35.2, 43.8] <
# (43.8, 52.4] < (52.4, 61.0]]
combine = [train_set, test_set]
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 26.6, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 26.6) & (dataset['Age'] <= 35.2), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 35.2) & (dataset['Age'] <= 43.8), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 43.8) & (dataset['Age'] <= 52.4), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 52.4, 'Age'] = 4
# train_set = train_set.drop(['AgeBand'], axis=1)

'''
# combine = [train_set, test_set] # 10/20대, 30/40대, 50/60대로 나누기
# for dataset in combine:    
#     dataset.loc[ dataset['Age'] <= 29, 'Age'] = 0
#     dataset.loc[(dataset['Age'] > 29) & (dataset['Age'] <= 39), 'Age'] = 1
#     dataset.loc[(dataset['Age'] > 39) & (dataset['Age'] <= 49), 'Age'] = 2
#     dataset.loc[ dataset['Age'] > 49, 'Age'] = 3


# combine = [train_set, test_set] # 10/20/30, 40, 50/60대로 나누기
# for dataset in combine:    
#     dataset.loc[ dataset['Age'] <= 39, 'Age'] = 0
#     dataset.loc[(dataset['Age'] > 39) & (dataset['Age'] <= 49), 'Age'] = 1
#     # dataset.loc[(dataset['Age'] > 49) & (dataset['Age'] <= 49), 'Age'] = 2
#     dataset.loc[ dataset['Age'] > 49, 'Age'] = 2

# combine = [train_set, test_set] # 10/20, 30, 40, 50, 60대로 나누기
# for dataset in combine:    
#     dataset.loc[ dataset['Age'] <= 29, 'Age'] = 0
#     dataset.loc[(dataset['Age'] > 29) & (dataset['Age'] <= 39), 'Age'] = 1
#     dataset.loc[(dataset['Age'] > 39) & (dataset['Age'] <= 49), 'Age'] = 2
#     dataset.loc[(dataset['Age'] > 49) & (dataset['Age'] <= 59), 'Age'] = 3
#     dataset.loc[ dataset['Age'] > 59, 'Age'] = 4

combine = [train_set, test_set] # 10/20, 30, 40, 50/60대로 나누기
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 29, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 29) & (dataset['Age'] <= 39), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 39) & (dataset['Age'] <= 49), 'Age'] = 3
    # dataset.loc[(dataset['Age'] > 49) & (dataset['Age'] <= 59), 'Age'] = 4
    dataset.loc[ dataset['Age'] > 49, 'Age'] = 4
'''

나이의 경우 범주화를 여러가지로 한 결과 순차적으로 5그룹으로 범주화시킨 경우가 가장 성능이 좋음을 확인했다.

In [ ]:
# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_set['TypeofContact'] = le.fit_transform(train_set['TypeofContact'])
test_set['TypeofContact'] = le.fit_transform(test_set['TypeofContact'])
train_set['Occupation'] = le.fit_transform(train_set['Occupation'])
test_set['Occupation'] = le.fit_transform(test_set['Occupation'])
train_set['Gender'] = le.fit_transform(train_set['Gender'])
test_set['Gender'] = le.fit_transform(test_set['Gender'])
train_set['ProductPitched'] = le.fit_transform(train_set['ProductPitched'])
test_set['ProductPitched'] = le.fit_transform(test_set['ProductPitched'])
train_set['MaritalStatus'] = le.fit_transform(train_set['MaritalStatus'])
test_set['MaritalStatus'] = le.fit_transform(test_set['MaritalStatus'])
train_set['Designation'] = le.fit_transform(train_set['Designation'])
test_set['Designation'] = le.fit_transform(test_set['Designation'])



from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
scaler = MinMaxScaler()
# scaler = StandardScaler()
# scaler = MaxAbsScaler()
# scaler = RobustScaler()

train_set[['Age', 'DurationOfPitch']] = scaler.fit_transform(train_set[['Age', 'DurationOfPitch']])
test_set[['Age', 'DurationOfPitch']] = scaler.transform(test_set[['Age', 'DurationOfPitch']])


# 모든 데이터 처리 완료 확인
# print(train_set.info())

train_set = train_set.drop(['NumberOfChildrenVisiting','NumberOfPersonVisiting',
                          'OwnCar', 
                          'MonthlyIncome', 
                          'NumberOfFollowups', 
                          'NumberOfTrips',
                          'NumberOfFollowups',
                          ], axis=1)
test_set = test_set.drop(['NumberOfChildrenVisiting','NumberOfPersonVisiting',
                          'OwnCar', 
                          'MonthlyIncome', 
                          'NumberOfFollowups', 
                          'NumberOfTrips',
                          'NumberOfFollowups',
                          ], axis=1)


x = train_set.drop(['ProdTaken'],axis=1) #axis는 컬럼 
# print(x) 

y = train_set['ProdTaken']
# print(y.shape)

# print(train_set.columns)

범주형 데이터를 레이블 인코딩해주고, 수치의 편차가 큰 데이터에 한해 스케일링을 진행하여 모든 데이터의 전처리를 완료했다.

상관관계가 적거나 포함 시에 성능이 떨어지는 컬럼을 삭제했다.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold


x_train, x_test, y_train, y_test = train_test_split(
  x, y, train_size=0.93, shuffle=True, random_state=1234, stratify=y)

n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=666)


# 최적의 매개변수
parameters = {'n_estimators' : [1000],
              'learning_rate' : [0.01],
              'depth': [15],
              'l2_leaf_reg' : [1],
              'model_size_reg' : [0],
              'od_pval' : [0],
}

# 2. model
from catboost import CatBoostClassifier
cat = CatBoostClassifier(random_state=123, verbose=False, n_estimators=500)
model = RandomizedSearchCV(cat, parameters, cv=kfold, n_jobs=-1)

#3. train, predict
model.fit(x_train, y_train)   
y_predict = model.predict(x_test)
from sklearn.metrics import accuracy_score
results = accuracy_score(y_test, y_predict)
print('최적의 매개변수 : ', model.best_params_)
print('최상의 점수 : ', model.best_score_)
print('acc :', results)
# print('걸린 시간 :', end_time)


#4. submission
y_summit = model.predict(test_set)
submission = pd.read_csv(path + 'sample_submission.csv')
submission['ProdTaken'] = y_summit
submission.to_csv(path + 'sample_submission_50_02.csv', index=False)



train set/test set을 나누어주고, StratifiedKFold 를 진행하여 교차검증을 진행하여 데이터를 최종적으로 준비한다.

캣부스트 모델에 랜덤서치를 이용하여 모델을 만들었고, 별도로 찾은 최적의 매개변수를 모델의 파라미터로 활용한다.